## 🚀 LangGraph Multi-Agent State 가이드
**"State 정의(TypedDict)는 데이터를 담을 '칸'을 만드는 설계도이고**

**실제 데이터의 이동은 부모와 자식 간의 '복사'와 '병합'을 통해 이루어진다!"**

### 1. "복사본(Copy)의 원칙" : 부모와 자식은 남남이다
- 복제 진입: 부모 그래프가 자식(서브그래프) 노드에 진입할 때, 부모의 State를 직접 건드리는 게 아니라 복사본을 만들어 자식에게 줍니다.
- 필터링: 복사할 때 무조건 다 가져오는 게 아니라, **자식 State(TypedDict) 설계도에 정의된 키(Key)**와 이름이 같은 것들만 골라 "값"을 복사해 줍니다.

### 2. "값(Value)이 들어와야 칸(Key)이 생긴다" : 물리적 생성의 원리
- 설계도 vs 실제 주머니: TypedDict에 필드를 정의한 것은 **'설계도'**일 뿐입니다. 실제 데이터 주머니에 그 필드(칸)가 생기는 시점은 **'값이 할당될 때'**입니다.
- KeyError의 정체: 설계도에는 필드가 있어도, 부모가 값을 안 줬고 자식 노드에서도 아직 값을 넣은 적이 없다면, 그 주머니엔 칸 자체가 없습니다. 그래서 state['key']를 하면 에러가 납니다.
- 해결책: state.get('key', 0)을 써서 칸이 없을 때의 초기값을 잡아주거나, 노드에서 return {"key": 0}을 통해 칸을 명시적으로 만들어줘야 합니다.

### 3. "데이터 병합(Merge)과 증발" : 돌아올 때의 검문
- 복귀: 자식 그래프가 끝나면 결과물을 부모 State에 합칩니다.
- 검문소: 이때 부모 State 설계도에 없는 키는 "받아줄 곳이 없다"고 판단되어 부모의 현재 상태에서는 증발합니다. (단, LangSmith 기록에는 남아있음!)
- 전략: 자식의 상세 과정(카운트 등)을 부모도 알게 하려면 부모 State에도 똑같은 이름의 필드를 정의하고, 최종 결과물만 필요하다면 answer 필드만 공유하면 됩니다.

### 4. "쌩뚱맞은 필드(설계도 외 키)" : 에러는 안 나지만 떠돌이 신세
- 유연성: 설계도에 없는 키를 노드에서 return 해도 파이썬 특성상 당장 에러가 나지는 않습니다.
- 리스크: 하지만 설계도에 없는 키는 LangGraph의 관리 대상이 아니므로, 노드 사이를 지나거나 부모로 돌아가는 과정에서 언제든 삭제될 수 있는 불안정한 데이터가 됩니다.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

small_llm = ChatOpenAI(model='gpt-4o-mini')

In [3]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    query: str
    context: list
    answer: str
    

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from typing import Literal
from langchain_core.output_parsers import StrOutputParser

def query_router(state: AgentState) -> Literal['income_tax', 'real_estate_tax','basic_generate']:
    query = state['query']

    query_router_system_prompt = """
    당신은 사용자의 질문을 'income_tax','real_estate_tax', 또는 'basic_generate'로 라우팅하는 전문가입니다.
    아래의 설명을 보고 사용자의 질문에 가장 적절한 경로를 선택해주세요.
    1. 'income_tax': 소득세 관련 정보가 필요한 질문에 대해서는 'income_tax'를 선택하세요. 
    2. 'real_estate_tax': 종합부동산세 관련 정보가 필요한 질문에 대해서는 'real_estate_tax'를 선택하세요.
    3. 'basic_generate': 사용자의 질문이 income_tax'와 'real_estate_tax' 모두 관련되지 않은 경우에는 'basic_generate'를 선택하세요. 
    중요: 답변은 무조건 income_tax, real_estate_tax, basic_generate 중 하나의 단어만 정확히 출력하세요. 다른 설명이나 문장은 절대 포함하지 마세요.
    """
    query_router_prompt = ChatPromptTemplate.from_messages([
        ("system", query_router_system_prompt),
        ("user", "{query}")
    ])

    query_router_chain = query_router_prompt | small_llm | StrOutputParser()

    route = query_router_chain.invoke({'query': query})

    print(f" [Router Decision] '{query}' 질문은 {route} 에이전트로 보냅니다. ")
    
    return route

In [5]:
def basic_generate(state: AgentState) -> AgentState:
    query = state['query']
    response = small_llm.invoke(query)
    return {'answer': response.content}

In [6]:
from income_tax_graph import graph as income_tax_agent
from real_estate_tax_graph import graph as real_estate_tax_agent

graph_builder = StateGraph(AgentState)

graph_builder.add_node("income_tax", income_tax_agent)
graph_builder.add_node("real_estate_tax", real_estate_tax_agent)
graph_builder.add_node("basic_generate", basic_generate)


/Users/lhj/inflearn-langgraph-agent/00-learning-path/real_estate_tax_graph.py:122: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily_search_tool = TavilySearchResults(


In [7]:
from langgraph.graph import START, END
graph_builder.add_conditional_edges(
    START,
    query_router,
    {
        "income_tax": "income_tax",
        "real_estate_tax": "real_estate_tax",
        "basic_generate": "basic_generate"
    }
)
graph_builder.add_edge("income_tax", END)
graph_builder.add_edge("real_estate_tax", END)
graph_builder.add_edge("basic_generate", END)

In [8]:
graph = graph_builder.compile()

In [ ]:
#initial_state = {'query': '소득세란 무엇인가요?'}
initial_state = {'query': '5000만원을 버는 직장인의 소득세는 얼마인가요?'}
graph.invoke(initial_state)

 [Router Decision] '5000만원을 버는 직장인의 소득세는 얼마인가요?' 질문은 income_tax 에이전트로 보냅니다. 


In [ ]:
initial_state = {'query': '15억짜리 집은 세금을 얼마나 내나요?'}
graph.invoke(initial_state)

In [ ]:
initial_state = {'query': '떡볶이가 맛있는 곳은 어디인가요?'}
graph.invoke(initial_state)
